$$\newcommand{\mean}[1]     {\langle #1 \rangle}$$
$$\newcommand{\spins}       {\mathbb{S}}$$
$$\newcommand{\spinsmu}     {\mathbb{S}_{\mu}}$$
$$\newcommand{\spini}       {s_{i}}$$
$$\newcommand{\spinj}       {s_{j}}$$
$$\newcommand{\Ham}         {\mathcal{H}}$$
$$\newcommand{\kB}          {k_B}$$

# Assignment 3: Ising model

In the lectures, we've been discussing the physics, some of the history, and the computational methods that underlie the analysis of the deceptively simple *Ising model*. In this assignment, you will bring much of that to bear by evaluating the evolution towards equilibrium of the Ising model system in 2 dimensions, and analyze the properties of the system once it reaches equilibrium. Given the extensive nature of this assigment, and the fact that it relies on much of what we have discussed so far, you have 2 weeks to complete it. That is, the assigments are due on *November 1st at 2pm* (via `GitHub` as usual).

I will include some of the background material that we discussed in lecture also here in order to keep this assignment somewhat self contained.

## The Ising Model

In seeking to explain a particular phenomenon in physics, namely the onset of ferromagnetism, Wilhelm Lenz proposed in 1924 that his PhD student, Ernst Ising, should solve a puzzling 1-dimensional model of  [[arXiv:1706.01764](https://arxiv.org/abs/1706.01764)]. This model was meant to attempt to describe the interaction of 

> "elementary magnetic units, which prefer alignment"

a problem which belonged to the new and undeveloped quantum mechanics. In order to make any progress at all, Ising started in 1 dimension with the task of calculating analytically the macroscopic magnetization with the methods of statistical mechanics. This deceptively *"simple"* model for a set of spin states $\mathbb{S} = \{s_i\}$, with a Hamiltonian, \Ham, for a given interaction coupling, $J$, and magnetic field, $H$, given by:

  \begin{equation}
    \Ham = -J \sum_{\mean{ij}}^N \spini \spinj - H \sum_i \spini "
  \end{equation} 

Given a set of $N$ spins, the Ising model has $2^N$ states $\mathbb{S} = \{s_i\}$ in $d=2$ dimensions. 

* What happens to the system when we let it evolve according to the laws of statistical mechanics? 
* Does the average spin remain random? 
* How does the application of an external field affect its evolution? 
* Can we calculate a specific heat for this system (the temperature change required to raise the system's energy by a given amount)?

The energy $E_{\mu}$ of a particular microstate $\mu$ of this (discrete) system is given by the operator \Ham acting on that microstate, or specific  $\spins = \spinsmu$. We assume that although the system is in an \bluebf{equilibrium state} (i.e. the energy of a particular element is proportional to the temperature, $T$), it is a dynamic one in which each element's energy fluctuates as it exchanges energy with its environment. The probability for the ensemble \spins to have energy $E_{\mu}$ is 
  
  \begin{equation}
    P(\spinsmu) = \frac{e^{-\beta E_{\mu}}}{\sum_{\mu} e^{-\beta E_{\mu}}},
  \end{equation} 

  And the mean and variance of the energy $\mean{E}$ (or *any* observable) is given by 
  
  \begin{eqnarray}
    \mean{E} &=& \sum_{\mu} E_{\mu} P(\spinsmu) = \frac{1}{Z} \sum_{\mu} E_{\mu} e^{-\beta E_{\mu}} \\
    Var(E) = \mean{(E-\mean{E}^2)^2} &=& \sum_{\mu} E^2_{\mu} P(\spinsmu) - \left( \sum_{\mu} E_{\mu} P(\spinsmu) \right)^2
  \end{eqnarray} 

##  Monte Carlo Methods

There is essentially only one known numerical method for calculating the partition function of a model such as the Ising model on a large lattice, and that method is *Monte Carlo simulation*. 
  
* If we are clever enough, we can obtain a *relatively good estimate* by only performing a *subset* of the calculations, $\{\mu\}$ instead of all $\mu$
* One way to be clever is to only sample the distribution that we are attempting to model, $P(\spinsmu)$ in regions where it is important.
* To put it another way, we want to perform a *weighted sampling*

  \begin{eqnarray}
    \mean{E} &=& \frac{\sum_{\{\mu\}} E_{\mu} e^{-\beta E_{\mu}} W_{\mu}^{-1}}{\sum_{\{\mu\}} e^{-\beta E_{\mu}} W_{\mu}^{-1}} \\
       &\approx& \frac{\sum_{\{\mu\}} E_{\mu}}{\sum_{\{\mu\}} 1} \\
       &=& \frac{1}{N^{\prime}}\sum_{\{\mu\}} E_{\mu}
  \end{eqnarray} 

  where $N^{\prime}$ is the number of terms in the subset $\{\mu\}$ and $W_{\mu} = e^{- \beta E_{\mu}}/Z$. Here, I have already set the weighting function to be the Boltzmann factor because of our discussion in lecture about the Metropolis algorithm.

## The Metropolis algorithm

In the famous paper *"Equation of State Calculations by Fast Computing Machines"*, Nicholas Metropolis, Arianna W. Rosenbluth, Marshall N. Rosenbluth, Augusta H. Teller, and Edward Teller proposed to use a Boltzmann factor for attempting the *trials* for a given change in configuration of the lattice.